In [3]:
# Imports here
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import optim, nn
from torchvision import datasets, transforms, models
from collections import OrderedDict
import torchvision
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from PIL import Image
import numpy as np

data_dir = 'drive/MyDrive/big_data/Cat_Dog_data'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'
class_names = ["cat", "dog"]

# Define your transforms for the training, validation, and testing sets

train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# Load the datasets with ImageFolder

train_data = torchvision.datasets.ImageFolder(train_dir , transform=train_transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=test_transform)

# Using the image datasets and the trainforms, define the dataloaders

trainloader = torch.utils.data.DataLoader(train_data, batch_size = 4, shuffle = True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle = False)
